<a href="https://colab.research.google.com/github/xBrymer/TensorFlow-Basics/blob/master/IMDB_yt_tuturial_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf



In [2]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEJ5R6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEJ5R6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEJ5R6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_dataset, test_dataset = dataset['train'], dataset['test']

encoder = info.features['text'].encoder



In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [5]:
padded_shapes = ([None], ())

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes = padded_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE,
                                         padded_shapes = padded_shapes
                                         )


In [27]:
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential


In [7]:

model = Sequential([
                    Embedding(encoder.vocab_size, 64),
                    Bidirectional(LSTM(64)),
                    Dense(64, activation = 'relu'),
                    Dense(1, activation = 'sigmoid')
])

In [11]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(1e-4),
    metrics = ['accuracy']
)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(train_dataset, epochs = 5, validation_data= test_dataset, validation_steps = 30)

Epoch 1/5
391/391 [==============================] - 45s 114ms/step - loss: 0.1803 - accuracy: 0.9390 - val_loss: 0.3352 - val_accuracy: 0.8547
Epoch 2/5
391/391 [==============================] - 44s 111ms/step - loss: 0.1599 - accuracy: 0.9476 - val_loss: 0.3610 - val_accuracy: 0.8620
Epoch 3/5
391/391 [==============================] - 44s 112ms/step - loss: 0.1442 - accuracy: 0.9532 - val_loss: 0.3656 - val_accuracy: 0.8615
Epoch 4/5
391/391 [==============================] - 43s 111ms/step - loss: 0.1317 - accuracy: 0.9582 - val_loss: 0.3733 - val_accuracy: 0.8599
Epoch 5/5
391/391 [==============================] - 44s 111ms/step - loss: 0.1203 - accuracy: 0.9628 - val_loss: 0.4193 - val_accuracy: 0.8625


In [37]:
def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sentence, model, pad):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text,64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return predictions



In [21]:
sample_text = ('This was awesome. The acting was incredible. Highly recommend')
predictions = sample_predict(sample_text, pad = True)*100

print('probability this is a positive review %.2f' % predictions)

sample_text = ('This move was so so. The acting was mediocre. Kind of')
predictions = sample_predict(sample_text, pad = True) * 100

print('probability this is a positive review %.2f' % predictions)

probability this is a positive review 92.86
probability this is a positive review 21.97


In [22]:

sample_text = ('This move is so fucking bad do not watch it')
predictions = sample_predict(sample_text, pad = True) * 100

print('probability this is a positive review %.2f' % predictions)

probability this is a positive review 28.85


In [28]:
model2 = Sequential([
                     Embedding(encoder.vocab_size, 64),
                     Bidirectional(LSTM(64, return_sequences = True)),
                     Bidirectional(LSTM(32)),
                     Dense(64, activation = 'relu'),
                     Dropout(0.5),
                     Dense(1, activation = 'sigmoid')
])



In [29]:
from tensorflow.keras.optimizers import Adam

In [33]:
model2.compile(loss = 'binary_crossentropy', optimizer = Adam(1e-4), metrics = ['accuracy'])

In [34]:
history2 = model2.fit(train_dataset, epochs =5, validation_data = test_dataset, validation_steps = 30)

Epoch 1/5
391/391 [==============================] - 78s 200ms/step - loss: 0.6750 - accuracy: 0.5576 - val_loss: 0.5842 - val_accuracy: 0.7240
Epoch 2/5
391/391 [==============================] - 78s 198ms/step - loss: 0.3832 - accuracy: 0.8439 - val_loss: 0.3668 - val_accuracy: 0.8401
Epoch 3/5
391/391 [==============================] - 80s 204ms/step - loss: 0.2660 - accuracy: 0.9042 - val_loss: 0.3603 - val_accuracy: 0.8479
Epoch 4/5
391/391 [==============================] - 79s 203ms/step - loss: 0.2133 - accuracy: 0.9286 - val_loss: 0.3767 - val_accuracy: 0.8615
Epoch 5/5
391/391 [==============================] - 78s 199ms/step - loss: 0.1777 - accuracy: 0.9460 - val_loss: 0.4050 - val_accuracy: 0.8599


In [38]:
sample_text = ('This was awesome. The acting was incredible. Highly recommend')
predictions = sample_predict(sample_text, model2, pad = True,)*100

print('probability this is a positive review %.2f' % predictions)

sample_text = ('This move was so so. The acting was mediocre. Kind of')
predictions = sample_predict(sample_text, model2, pad = True) * 100

print('probability this is a positive review %.2f' % predictions)

probability this is a positive review 92.68
probability this is a positive review 6.16


In [39]:
def predict_text(text, model):
  predictions = sample_predict(text, model, pad =True)*100
  print(text)
  print('probability this is a positive review %.2f' % predictions)
  return

In [46]:
predict_text("Best movie ever that sucks", model)

Best movie ever that sucks
probability this is a positive review 27.92
